**Convolution Neural Network**

In [8]:
import tensorflow as tf


In [9]:
# from IPython import get_ipython
# from IPython.display import display
# import tensorflow as tf

# import zipfile
# import io
# import os
# import zipfile
# import io
# import os
# zip_path = r"/path/to/your/zipfile/archive.zip"
# extract_dir = './extracted_data'
# os.makedirs(extract_dir, exist_ok=True)

# try:
#     with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#         zip_ref.extractall(extract_dir)
#     print(f"Successfully extracted {zip_path} to {extract_dir}")
# except FileNotFoundError:
#     print(f"Error: Zip file not found at {zip_path}")
# except Exception as e:
#     print(f"An error occurred during extraction: {e}")


In [27]:
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout
from keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import L2


In [28]:
cnn = Sequential()

In [29]:
cnn.add(Conv2D(filters=32,kernel_size=3,activation='relu',input_shape=[64,64,3]))
cnn.add(MaxPooling2D(pool_size=2,strides=2))
cnn.add(Conv2D(filters=16,kernel_size=3,activation='relu'))
cnn.add(MaxPooling2D(pool_size=2,strides=2))
cnn.add(Flatten())

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [30]:
cnn.add(Dense(64,activation='relu',kernel_regularizer=L2(0.01)))
cnn.add(BatchNormalization(),Dropout(0.2))
cnn.add(Dense(32,activation='relu'))
cnn.add(BatchNormalization(),Dropout(0.2))
cnn.add(Dense(16,activation='relu'))
cnn.add(BatchNormalization(),Dropout(0.2))
cnn.add(Dense(8,activation='relu'))
cnn.add(BatchNormalization(),Dropout(0.2))
cnn.add(Dense(4,activation='relu'))
cnn.add(BatchNormalization(),Dropout(0.2))
cnn.add(Dense(2,activation='relu'))
cnn.add(BatchNormalization(),Dropout(0.2))
cnn.add(Dense(1,activation='sigmoid'))

In [31]:
cnn.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [32]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import zipfile
import os
from google.colab import files

# uploaded = files.upload()

zip_path = '/content/archive.zip'

extract_dir = '/content/extracted_data'
os.makedirs(extract_dir, exist_ok=True)

try:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"Successfully extracted {zip_path} to {extract_dir}")
except FileNotFoundError:
    print(f"Error: Zip file not found at {zip_path}")
    exit()
except Exception as e:
    print(f"An error occurred during extraction: {e}")
    exit()

cnn = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_directory_path = os.path.join(extract_dir, 'train')
test_directory_path = os.path.join(extract_dir, 'test')

if not os.path.exists(train_directory_path):
    print(f"Error: Training directory not found: {train_directory_path}")
    exit()
elif not os.path.exists(test_directory_path):
    print(f"Error: Testing directory not found: {test_directory_path}")
    exit()

train_generator = train_datagen.flow_from_directory(
    train_directory_path,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_directory_path,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = test_generator.samples // test_generator.batch_size

cnn.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=10,
    validation_data=test_generator,
    validation_steps=validation_steps
)


test_loss, test_accuracy = cnn.evaluate(test_generator)
print(f"Test accuracy: {test_accuracy:.4f}")

cnn.save('/content/cat_dog_classifier.h5')

Successfully extracted /content/archive.zip to /content/extracted_data
Found 557 images belonging to 2 classes.
Found 140 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 10s 495ms/step - accuracy: 0.5540 - loss: 0.7454 - val_accuracy: 0.4922 - val_loss: 0.7230
Epoch 2/10
 1/17 ━━━━━━━━━━━━━━━━━━━━ 2s 160ms/step - accuracy: 0.4688 - loss: 0.7141

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - accuracy: 0.4688 - loss: 0.7141 - val_accuracy: 0.5078 - val_loss: 0.7076
Epoch 3/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 7s 386ms/step - accuracy: 0.5291 - loss: 0.6950 - val_accuracy: 0.4688 - val_loss: 0.7018
Epoch 4/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - accuracy: 0.4062 - loss: 0.7091 - val_accuracy: 0.5156 - val_loss: 0.6881
Epoch 5/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 10s 574ms/step - accuracy: 0.6006 - loss: 0.6711 - val_accuracy: 0.5625 - val_loss: 0.6900
Epoch 6/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - accuracy: 0.7500 - loss: 0.6707 - val_accuracy: 0.5391 - val_loss: 0.7283
Epoch 7/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 10s 589ms/step - accuracy: 0.5740 - loss: 0.6771 - val_accuracy: 0.5859 - val_loss: 0.6709
Epoch 8/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - accuracy: 0.6562 - loss: 0.6311 - val_accuracy: 0.5859 - val_loss: 0.6829
Epoch 9/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 20s 676ms/step - accuracy: 0.6318 - loss: 0.6355 - val_accuracy: 0.5078 - val_l

Test accuracy: 0.5214
